In [201]:
!pip install transformers
!pip install torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}+${CUDA}.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-+.html


In [202]:
!pip install torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}+${CUDA}.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-+.html


In [203]:
!pip install torch-geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [204]:
from transformers import RobertaConfig, RobertaTokenizer, RobertaForMaskedLM, pipeline

CBmodel = RobertaForMaskedLM.from_pretrained("microsoft/codebert-base-mlm", output_hidden_states = True)
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base-mlm")
# https://huggingface.co/docs/transformers/model_doc/roberta#transformers.RobertaForMaskedLM

TODO: CREATE A GENERATOR(FILES) METHOD FOR graph_dump.txt FILES GENERATED BY THE PDG

In [205]:
files = ["/content/sample_data/example1_graph_dump.txt", "/content/sample_data/example2_graph_dump.txt", "/content/sample_data/AnishaSample_graph_dump.txt"]

In [206]:
from torch import nn
import torch
!python -c "import torch; print(torch.__version__)"
!python -c "import torch; print(torch.version.cuda)"
import platform
print(platform.python_version())
from typing import Callable, List, Optional, Dict
import numpy as np
import scipy.sparse as sp

1.12.0+cu113
11.3
3.7.13


DOUBT!!!!! SHOULD WE COUNT NODES PER GRAPH OR ACROSS ALL GRAPHS??CURRENTLY COUNTING PER GRAPH.

In [207]:
# nodes_to_graph_id = []
def get_nodes_edges(inTextFile):
  # FD = 0, CD = 1
  # to support the hetero data object as suggested by the documentation 
  nodes_dict = {}
  edge_indices_CD = []
  edge_indices_FD = []

  #to support the Data object as used by the Entities dat object as used in RGAT source code
  edge_indices = []
  edge_type = []
  
  
  # nodes_dict is an index_map
  node_count=0;
  with open(inTextFile) as fp:

    Lines = fp.readlines()

    for line in Lines:

      N = line.split('-->')
        
      t1 = N[0].split('$$')   
      src = t1[1].strip()   
      if src not in nodes_dict.keys():
        nodes_dict[src] = node_count
        # nodes_to_graph_id.append(graph_id)
        node_count+=1
        
      t2 = N[1].split('$$')
      idx = t2[1].rfind('[')
      dst = t2[1][:idx].strip()
      if dst not in nodes_dict.keys():
        nodes_dict[dst] = i
        # nodes_to_graph_id.append(graph_id)
        node_count+=1

      x = t2[1][idx+2:-3]
      if(x=='FD'):
        y=0
        edge_indices.append([nodes_dict[src], nodes_dict[dst]])
        edge_type.append(y)
        edge_indices_FD.append([nodes_dict[src], nodes_dict[dst]])
      else: 
        y=1
        edge_type.append(y)
        edge_indices.append([nodes_dict[src], nodes_dict[dst]])
        edge_indices_CD.append([nodes_dict[src], nodes_dict[dst]])
     
  # return nodes_to_graph_id, j, nodes_dict, edge_indices_FD, edge_indices_CD
  return nodes_dict, edge_indices_FD, edge_indices_CD, edge_indices, edge_type

In [208]:
def get_nodes_feat_CodeBert(nodes_dict , graph_id, i):
  # Stores the token vectors, with shape [22 x 768]
  CodeEmbedding = []
  for text in nodes_dict.keys():
    # print(text)
    i = nodes_dict[text]
    T=30
    tokens=tokenizer.tokenize(str(text))
    tokens = ['[CLS]'] + tokens + ['[SEP]']
    padded_tokens=tokens +['[PAD]' for _ in range(T-len(tokens))]
    attn_mask=[ 1 if token != '[PAD]' else 0 for token in padded_tokens  ]
    seg_ids=[0 for _ in range(len(padded_tokens))]
    sent_ids=tokenizer.convert_tokens_to_ids(padded_tokens)
    token_ids = torch.tensor(sent_ids).unsqueeze(0) 
    attn_mask = torch.tensor(attn_mask).unsqueeze(0) 
    seg_ids   = torch.tensor(seg_ids).unsqueeze(0)
    with torch.no_grad():
      output = CBmodel(token_ids, attention_mask = attn_mask, token_type_ids = seg_ids)
      hid_state = output.hidden_states[-1][0]
      cls_embedd = hid_state[0]
      # print(tf.shape(cls_embedd))
      # print(cls_embedd)
      m = nn.Linear(768, 64)
      se = m(cls_embedd)
      # print(tf.shape(se))
      CodeEmbedding.append(se.tolist())
      # token_vecs_sum = torch.cat((token_vecs_sum, token_vecs))
      
      # print(i, str(CodeEmbedding[i]))
  return CodeEmbedding

In [209]:
batchsize = 3

from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import RGATConv, to_hetero
# from torch_geometric.nn import global_add_pool
# from torch_geometric.nn import global_sort_pool
from torch_geometric.nn import global_max_pool

class RGAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels,
                 num_relations):
        super().__init__()
        torch.manual_seed(12345)
        self.conv1 = RGATConv(in_channels, hidden_channels, num_relations)
        self.conv2 = RGATConv(hidden_channels, hidden_channels, num_relations)
        self.lin = torch.nn.Linear(hidden_channels, out_channels)

    def forward(self, x, edge_index, edge_type, batch):
        x = self.conv1(x, edge_index, edge_type).relu()
        x = self.conv2(x, edge_index, edge_type)

        # 2. Readout layer
        x = global_max_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x
        

In [210]:
from torch_geometric.data import (
    HeteroData,
    Data,
    InMemoryDataset,
    Batch
    )
# from torch_geometric.typing import EdgeType, EdgeTensorType
from torch_geometric.data.storage import EdgeStorage
import torch_geometric.datasets as datasets
import torch_geometric.transforms as transforms
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

In [211]:
# Hetero data
data_list = []
# Data
d_list =[]
for i, file in enumerate(files):
  j=0;
  # GET DATA
  # nodes_dict = {}
  # nodes_to_graph_id = []
  nodes_dict, edge_indices_FD, edge_indices_CD, edge_indices, edge_type = get_nodes_edges(file)
  # print(nodes_dict)
  # print(edge_indices_CD)
  # print(edge_indices_FD)

  # Node feature matrix with shape [num_nodes, num_node_features]=(6,64).
  CodeEmbedding = get_nodes_feat_CodeBert(nodes_dict, 0, 0)
   # print(CodeEmbedding)

  # FIXING DATA FOTMATS AND SHAPE
  x = torch.tensor(CodeEmbedding)
  # print(x.shape)
  # -[6,64]
  
  # data.y: Target to train against (may have arbitrary shape),
  # graph-level targets of shape [1, *]
  y=torch.tensor([0.0,0.0,1.0]).reshape([1, 3])
  print(type(y))

 # edge_index (LongTensor, optional) – Graph connectivity in COO format with shape [2, num_edges]
  edge_index_CD = torch.tensor(edge_indices_CD, dtype=torch.long).t().contiguous()
  # print(edge_index_CD)
  edge_index_FD = torch.tensor(edge_indices_FD, dtype=torch.long).t().contiguous()
  # print(edge_index_FD)
  edge_index = torch.tensor(edge_indices, dtype=torch.long).t().contiguous()
  edge_type = torch.tensor(edge_type, dtype=torch.long).t().contiguous()
  # INITIALIZING A Hetero DATA POINT - documentation based
  hdata = HeteroData(node={ 'x': x })
  hdata['node', 'CD', 'node'].edge_index = edge_index_CD
  hdata['node', 'FD', 'node'].edge_index = edge_index_FD
  hdata.y = y
  hdata['node'].num_nodes = len(nodes_dict)
  hdata['node'].num_node_features = len(CodeEmbedding[0])
  node_types, edge_types = hdata.metadata()
  # print(edge_types)
  # MAKE A DATALIST
  data_list.append(hdata)

  data = Data(edge_index=edge_index, edge_type=edge_type, x=x)
  data.y = y;
  data.num_nodes = len(nodes_dict)
  d_list.append(data)






<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>


In [212]:
loader = DataLoader(data_list, batch_size=2)

In [213]:
loader2 = DataLoader(d_list, batch_size=2)

In [214]:
vars(loader)

{'_DataLoader__initialized': True,
 '_DataLoader__multiprocessing_context': None,
 '_IterableDataset_len_called': None,
 '_dataset_kind': 0,
 '_iterator': None,
 'batch_sampler': <torch.utils.data.sampler.BatchSampler at 0x7f1aed740110>,
 'batch_size': 2,
 'collate_fn': <torch_geometric.loader.dataloader.Collater at 0x7f1aed740310>,
 'dataset': [HeteroData(
    y=[1, 3],
    node={
      x=[6, 64],
      num_nodes=6,
      num_node_features=64
    },
    (node, CD, node)={ edge_index=[2, 5] },
    (node, FD, node)={ edge_index=[2, 5] }
  ), HeteroData(
    y=[1, 3],
    node={
      x=[6, 64],
      num_nodes=6,
      num_node_features=64
    },
    (node, CD, node)={ edge_index=[2, 4] },
    (node, FD, node)={ edge_index=[2, 6] }
  ), HeteroData(
    y=[1, 3],
    node={
      x=[6, 64],
      num_nodes=6,
      num_node_features=64
    },
    (node, CD, node)={ edge_index=[2, 5] },
    (node, FD, node)={ edge_index=[2, 2] }
  )],
 'drop_last': False,
 'exclude_keys': None,
 'follow_b

In [215]:
vars(loader2)

{'_DataLoader__initialized': True,
 '_DataLoader__multiprocessing_context': None,
 '_IterableDataset_len_called': None,
 '_dataset_kind': 0,
 '_iterator': None,
 'batch_sampler': <torch.utils.data.sampler.BatchSampler at 0x7f1aed7be350>,
 'batch_size': 2,
 'collate_fn': <torch_geometric.loader.dataloader.Collater at 0x7f1aed7bed50>,
 'dataset': [Data(x=[6, 64], edge_index=[2, 10], edge_type=[10], y=[1, 3], num_nodes=6),
  Data(x=[6, 64], edge_index=[2, 10], edge_type=[10], y=[1, 3], num_nodes=6),
  Data(x=[6, 64], edge_index=[2, 7], edge_type=[7], y=[1, 3], num_nodes=6)],
 'drop_last': False,
 'exclude_keys': None,
 'follow_batch': None,
 'generator': None,
 'num_workers': 0,
 'persistent_workers': False,
 'pin_memory': False,
 'pin_memory_device': '',
 'prefetch_factor': 2,
 'sampler': <torch.utils.data.sampler.SequentialSampler at 0x7f1aed7be1d0>,
 'timeout': 0,
 'worker_init_fn': None}

In [216]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  #num_classes??3 for now
  #num_relations?? 2 for now
model = RGAT(64, 16, 3, 2).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.0005)
criterion = torch.nn.CrossEntropyLoss()
# model = to_hetero(model, data.metadata(), aggr='sum')

In [217]:
model

RGAT(
  (conv1): RGATConv(64, 16, heads=1)
  (conv2): RGATConv(16, 16, heads=1)
  (lin): Linear(in_features=16, out_features=3, bias=True)
)

In [218]:
for data in loader:
  print(vars(data))
  # data = data.to(device)
  print()
  print(data._edge_store_dict)

{'_global_store': {'y': tensor([[0., 0., 1.],
        [0., 0., 1.]])}, '_node_store_dict': {'node': {'x': tensor([[ 2.9443e-01,  2.4020e-01, -3.5007e-02, -5.3109e-01,  6.1027e-03,
         -1.6022e-01, -3.8866e-01,  2.0816e-01,  3.0737e-01,  7.0874e-02,
         -3.3227e-01,  1.6522e-01,  3.9254e-01, -9.6531e-02, -1.2251e-01,
          2.7382e-01, -4.3338e-01, -2.6536e-01, -3.2666e-01, -1.2912e-01,
         -1.4889e-01, -1.8789e-01,  3.2412e-01, -4.8672e-01,  4.2398e-01,
         -2.0261e-01,  1.1859e-01, -1.8840e-01, -3.6493e-01,  4.5412e-01,
          8.1351e-02, -4.3888e-01, -2.8010e-01,  1.9178e-01,  4.0664e-01,
         -1.9875e-01, -2.1171e-02, -3.4211e-01,  8.3194e-02,  7.9346e-02,
         -1.6098e-02, -3.1080e-01, -5.3192e-01,  5.1327e-01, -4.0966e-01,
          1.0375e-01, -2.8884e-01,  2.5118e-02,  1.7064e-01,  1.1391e-01,
          3.4254e-01,  1.1217e-01, -5.7126e-01,  2.9078e-01, -1.6091e-01,
          2.4829e-02, -5.0029e-01, -2.8908e-02, -1.7337e-01,  1.7868e-01,
      

In [219]:
for data in loader2:
  print(vars(data))
  # data = data.to(device)
  print(data._store.x)
  print(data.edge_index)
  print(data.edge_type)
  print(data.batch)

{'_store': {'x': tensor([[ 2.9443e-01,  2.4020e-01, -3.5007e-02, -5.3109e-01,  6.1027e-03,
         -1.6022e-01, -3.8866e-01,  2.0816e-01,  3.0737e-01,  7.0874e-02,
         -3.3227e-01,  1.6522e-01,  3.9254e-01, -9.6531e-02, -1.2251e-01,
          2.7382e-01, -4.3338e-01, -2.6536e-01, -3.2666e-01, -1.2912e-01,
         -1.4889e-01, -1.8789e-01,  3.2412e-01, -4.8672e-01,  4.2398e-01,
         -2.0261e-01,  1.1859e-01, -1.8840e-01, -3.6493e-01,  4.5412e-01,
          8.1351e-02, -4.3888e-01, -2.8010e-01,  1.9178e-01,  4.0664e-01,
         -1.9875e-01, -2.1171e-02, -3.4211e-01,  8.3194e-02,  7.9346e-02,
         -1.6098e-02, -3.1080e-01, -5.3192e-01,  5.1327e-01, -4.0966e-01,
          1.0375e-01, -2.8884e-01,  2.5118e-02,  1.7064e-01,  1.1391e-01,
          3.4254e-01,  1.1217e-01, -5.7126e-01,  2.9078e-01, -1.6091e-01,
          2.4829e-02, -5.0029e-01, -2.8908e-02, -1.7337e-01,  1.7868e-01,
         -1.5459e-02,  3.1262e-01,  2.0956e-01, -2.5593e-01],
        [ 1.7770e-01, -2.3009e-01

In [238]:
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data._store.x, data.edge_index, data.edge_type, data.batch)
    loss = criterion(out, data.y)  # Compute the loss.
    loss.backward()  # Derive gradients.
    optimizer.step()  # Update parameters based on gradients.
    print(loss)
    return loss

In [221]:
# batch = Batch().from_data_list(data_list)

In [222]:
# vars(batch)

In [223]:
# len(batch._edge_store_dict)

In [224]:
# from torch_geometric.loader import DataLoader

# train_loader = DataLoader(batch, batch_size=1, shuffle=True)
# test_loader = DataLoader(batch, batch_size=1, shuffle=False)

In [242]:
@torch.no_grad()
def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.
         out = model(data.x, data.edge_index, data.edge_type,data.batch)  
         pred = out.argmax(dim=1)  # Use the class with highest probability.
         correct += int((pred == data.y.argmax(dim=1)).sum())  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


In [226]:
# model = RGAT(data.num_features, 16, data.num_classes, data.num_relations)
# print(model)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.0005)


In [227]:
# data = process0(x=x,y=y,edge_indices=edge_indices, edge_type=edge_type, edge_index1=edge_indices1, edge_index0=edge_indices0)
# data = process1(x=x,y=y,edge_index=edge_index, edge_type=edge_type)

In [240]:
train()

tensor(0.1387, grad_fn=<DivBackward1>)


tensor(0.1387, grad_fn=<DivBackward1>)

In [243]:
for epoch in range(1, 2):
    train()
    train_acc = test(loader2)
    test_acc = test(loader2)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

tensor(0.5359, grad_fn=<DivBackward1>)
Epoch: 001, Train Acc: 1.0000, Test Acc: 1.0000
